In [1]:
"""
의사결정트리:오버피팅->가지치기=>부족한 알고리즘
랜덤포레스트:의사결정트리의 한계를 극복하기 위한 알고리즘
나무 생성? 배깅
EX) 학습 데이터 1000개(행), 25개(속성) -> 임의로 100개의 데이터 선택(31번 반복) => 트리(31개) 구성
->트리 작성시 사용될 피처(features)들을 제한 -> 나무에 대한 다양성
=>속성 몇개? "전체 속성 개수(25개)의 제곱근이 가장 좋다" -> 5개를 속성 개수로 사용

*중요 옵션*
n_estimators:트리의 개수(기본:100), 크게 할수록 좋음(시간은 오래 걸림)
min_samples_split:노드를 분할하기 위한 최소한의 데이터수(과적합을 제어),
작게 설정할수록 분할 노드가 많아지므로 과적합이 증가할 수 있음.
max_depth:트리의 최대 깊이 (기본:None, 완전하게 클래스값이 결정될때까지 분할)
또는 데이터 개수가 min_samples_split보다 작아질때까지 분할
min_samples_leaf:리프노드(터미널노드)가 되기 위해 필요로하는 최소한의 샘플 데이터수.
일반적으로 작게설정
=> 최적의 랜덤포레스트 퍼러미터를 설정하는게 중요!!!(과거(현재)에는 수동)->
GridSearchCV를 사용해서 랜덤포레스트 하이퍼 퍼러미터 튜닝


5  * 5  *10 *  3=750
GridSearchCV 설정 예)
myparam={
n_estimators:[5, 10, 30, 50, 100, 200],
min_samples_split:[3, 5, ...]
...
}

colab 사용 추천(무료)



AI:1940대~(튜링머신)
<-----------머신러닝--------------->
신경망(단일퍼셉트론->멀티퍼셉트론)/DT/... -> 랜덤포레스트/SVM -> 딥러닝(깊은신경망)


"""

'\n의사결정트리:오버피팅->가지치기=>부족한 알고리즘\n랜덤포레스트:의사결정트리의 한계를 극복하기 위한 알고리즘\n나무 생성? 배깅\nEX) 학습 데이터 1000개(행), 25개(속성) -> 임의로 100개의 데이터 선택(31번 반복) => 트리(31개) 구성\n->트리 작성시 사용될 피처(features)들을 제한 -> 나무에 대한 다양성\n=>속성 몇개? "전체 속성 개수(25개)의 제곱근이 가장 좋다" -> 5개를 속성 개수로 사용\n\n*중요 옵션*\nn_estimators:트리의 개수(기본:100), 크게 할수록 좋음(시간은 오래 걸림)\nmin_samples_split:노드를 분할하기 위한 최소한의 데이터수(과적합을 제어),\n작게 설정할수록 분할 노드가 많아지므로 과적합이 증가할 수 있음.\nmax_depth:트리의 최대 깊이 (기본:None, 완전하게 클래스값이 결정될때까지 분할)\n또는 데이터 개수가 min_samples_split보다 작아질때까지 분할\nmin_samples_leaf:리프노드(터미널노드)가 되기 위해 필요로하는 최소한의 샘플 데이터수.\n일반적으로 작게설정\n=> 최적의 랜덤포레스트 퍼러미터를 설정하는게 중요!!!(과거(현재)에는 수동)->\nGridSearchCV를 사용해서 랜덤포레스트 하이퍼 퍼러미터 튜닝\n\n\n5  * 5  *10 *  3=750\nGridSearchCV 설정 예)\nmyparam={\nn_estimators:[5, 10, 30, 50, 100, 200],\nmin_samples_split:[3, 5, ...]\n...\n}\n\ncolab 사용 추천(무료)\n\n\n\nAI:1940대~(튜링머신)\n<-----------머신러닝--------------->\n신경망(단일퍼셉트론->멀티퍼셉트론)/DT/... -> 랜덤포레스트/SVM -> 딥러닝(깊은신경망)\n\n\n'

In [158]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [159]:
train=pd.read_csv("train.csv", parse_dates=['datetime'])
#train.info()
test=pd.read_csv("test.csv", parse_dates=['datetime'])
#test.info()
#print(train['datetime'])
train['year']=train['datetime'].dt.year
train['month']=train['datetime'].dt.month
train['day']=train['datetime'].dt.day
train['hour']=train['datetime'].dt.hour
train['minute']=train['datetime'].dt.minute
train['second']=train['datetime'].dt.second
train['dayofweek']=train['datetime'].dt.dayofweek

test['year']=test['datetime'].dt.year
test['month']=test['datetime'].dt.month
test['day']=test['datetime'].dt.day
test['hour']=test['datetime'].dt.hour
test['minute']=test['datetime'].dt.minute
test['second']=test['datetime'].dt.second
test['dayofweek']=test['datetime'].dt.dayofweek

In [160]:
test.shape #(6493, 16)
train.shape #(10886, 19)


(10886, 19)

In [161]:
#1. 퀴즈
#풍속(windspeed)의 0값을 풍속의 평균값으로 모두 대체하라
#=train['windspeed'].mean()
#train.loc[train['windspeed']==0]['windspeed']=train['windspeed'].mean()
#train.loc[train['windspeed']==0, 'windspeed']=train['windspeed'].mean()

#좋은 방법이 아님

In [162]:
#머신러닝(랜덤포레스트)를 이용한 풍속 0 값을 대체
#풍속 0, not 0으로 분류
trainWind0=train.loc[train.windspeed==0]
trainWindNot0=train.loc[train.windspeed!=0]
print(trainWind0.shape) #1313,19
#print(trainWindNot0)

(1313, 19)


In [163]:
from sklearn.ensemble import RandomForestClassifier

In [164]:
train.columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count',
       'year', 'month', 'day', 'hour', 'minute', 'second', 'dayofweek'],
      dtype='object')

In [165]:
def predictWindSpeed(data):
    #data의 windspeed열 값 0을 랜덤포레스트 기반 예측값으로 대체
    dataWind0=data.loc[data['windspeed']==0]
    dataWindNot0=data.loc[data['windspeed']!=0]
    
    #입력데이터() -> 랜덤포레스트 모델 -> 출력데이터(windspeed)
    #풍속을 예측하는데 사용될 변수(입력)를 선택
    wCol=['season','weather', 'temp', 'atemp','humidity','year','month']
    
    #회귀모델
    #풍속예측함수 = w1*season+w2*weather+...+w7*month+b    
    
    #출력 데이터 타입이 str이어야 함.
    dataWindNot0['windspeed']=dataWindNot0['windspeed'].astype('str')
    
    #랜덤포레스트 분류기
    rfModelWind=RandomForestClassifier()
    
    #wind not 0 데이터로 학습(fit)을 시켜서 모델을 만든 후, wind 0 데이터를 모델에
    #입력하면 예상되는 풍속이 출력된다   
    #모델링(학습데이터)
    rfModelWind.fit(dataWindNot0[wCol], dataWindNot0['windspeed'])
    
    #모델(rfModelWind)을 이용하여 풍속이 0인 데이터에 대한 풍속을 예측(predict)하자.
    wind0Values=rfModelWind.predict(dataWind0[wCol])
    
    predictWind0=dataWind0
    predictWindNot0=dataWindNot0
    
    #풍속 0을 예측된 값으로 변경
    predictWind0['windspeed']=wind0Values
    
    #풍속이 0이 아닌 데이터프레임에 예측된 값이 저장된 데이터프레임을 합침
    data=predictWindNot0.append(predictWind0)
    
    ################누락#################
    #타입 변경 안했음 -> 변경!!!
    data['windspeed']=data['windspeed'].astype('float')
    
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data
    

In [166]:
train=predictWindSpeed(train)
#test=predictWindSpeed(test)

<ipython-input-165-99d6381bea3e>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataWindNot0['windspeed']=dataWindNot0['windspeed'].astype('str')
<ipython-input-165-99d6381bea3e>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictWind0['windspeed']=wind0Values


In [167]:
print(train[train.windspeed==0])
print(test[test.windspeed==0])


Empty DataFrame
Columns: [datetime, season, holiday, workingday, weather, temp, atemp, humidity, windspeed, casual, registered, count, year, month, day, hour, minute, second, dayofweek]
Index: []
                datetime  season  holiday  workingday  weather   temp   atemp  \
1    2011-01-20 01:00:00       1        0           1        1  10.66  13.635   
2    2011-01-20 02:00:00       1        0           1        1  10.66  13.635   
12   2011-01-20 12:00:00       1        0           1        2  12.30  16.665   
60   2011-01-22 13:00:00       1        0           0        1   3.28   8.335   
62   2011-01-22 15:00:00       1        0           0        1   4.92   7.575   
...                  ...     ...      ...         ...      ...    ...     ...   
6433 2012-12-29 12:00:00       1        0           0        3   8.20  12.120   
6434 2012-12-29 13:00:00       1        0           0        3   8.20  12.120   
6436 2012-12-29 15:00:00       1        0           0        2   9.84  12.1

In [168]:
train['windspeed'].describe()

count    10886.000000
mean        14.010535
std          7.041554
min          6.003200
25%          8.998100
50%         12.998000
75%         19.001200
max         56.996900
Name: windspeed, dtype: float64

In [169]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10886 non-null  datetime64[ns]
 1   season      10886 non-null  int64         
 2   holiday     10886 non-null  int64         
 3   workingday  10886 non-null  int64         
 4   weather     10886 non-null  int64         
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   casual      10886 non-null  int64         
 10  registered  10886 non-null  int64         
 11  count       10886 non-null  int64         
 12  year        10886 non-null  int64         
 13  month       10886 non-null  int64         
 14  day         10886 non-null  int64         
 15  hour        10886 non-null  int64         
 16  minute      10886 non-

In [170]:
#연속형 변수 -> 범주형 변수
category_fn=['season','holiday','workingday','weather',
            'year', 'month', 'hour', 'dayofweek']

In [171]:
for v in category_fn:
    train[v]=train[v].astype('category')
    test[v]=test[v].astype('category')

In [172]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10886 non-null  datetime64[ns]
 1   season      10886 non-null  category      
 2   holiday     10886 non-null  category      
 3   workingday  10886 non-null  category      
 4   weather     10886 non-null  category      
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   casual      10886 non-null  int64         
 10  registered  10886 non-null  int64         
 11  count       10886 non-null  int64         
 12  year        10886 non-null  category      
 13  month       10886 non-null  category      
 14  day         10886 non-null  int64         
 15  hour        10886 non-null  category      
 16  minute      10886 non-

In [173]:
from sklearn.metrics import make_scorer

In [174]:
def rmsle(pv, av): #예측값, 실제값
    #넘파이 배열로 변환
    pv=np.array(pv)
    av=np.array(av)
    
    #예측값과 실제값에 1을 더하고 로그를 씌운다
    log_predict=np.log(pv+1)
    log_actual=np.log(av+1)
    
    res=log_predict-log_actual
    res=np.square(res)
    
    mean_res=res.mean()
    score=np.sqrt(mean_res)
    return score  
    

In [175]:
rmsle_scorer=make_scorer(rmsle)
rmsle_scorer

make_scorer(rmsle)

In [ ]:
score=cross_val_score(model, xTrain, yTrain, cv=kfold, scoring=rmsle_scorer)
score
score.mean() #0.32

# xTrain이 model에 들어가면 예측결과가 나옴(pv) ->rmsle함수에 전달
# yTrain(av) -> rmsle함수에 전달
# => score 리턴
# 위 과정이 10번 반복(kfold=10), score 10번 리턴



In [176]:
"""

데이터 100건 -> 70건(트레이닝)=>모델, 30건(테스트)

교차검증? 모델의 일반화 성능을 측정하기 위해, 
데이터를 여러 겹(fold)으로 나누고, 트레이닝/테스트 용으로 나뉘어진 폴드를 
다양하게 적용하여 모델을 학습하고, 평가

100건 데이터, 4겹(fold, 1겹당 25건 데이터), 
첫번째 fold(테스트용, 25건), 두번째~네번째 fold(트레이닝용, 25*3=75건) : 정확도
두번째 fold(테스트용, 25건), 첫번째, 세,네번째 fold(트레이닝용, 25*3=75건) : 정확도
세번째 fold(테스트용, 25건), 첫,두번째, 네번째 fold(트레이닝용, 25*3=75건) : 정확도
네번째 fold(테스트용, 25건), 첫~세번째 fold(트레이닝용, 25*3=75건) :정확도

정확도의 평균(최종 모델 정확도)

kfold 교차검증
"""

'\n\n데이터 100건 -> 70건(트레이닝)=>모델, 30건(테스트)\n\n교차검증? 모델의 일반화 성능을 측정하기 위해, \n데이터를 여러 겹(fold)으로 나누고, 트레이닝/테스트 용으로 나뉘어진 폴드를 \n다양하게 적용하여 모델을 학습하고, 평가\n\n100건 데이터, 4겹(fold, 1겹당 25건 데이터), \n첫번째 fold(테스트용, 25건), 두번째~네번째 fold(트레이닝용, 25*3=75건) : 정확도\n두번째 fold(테스트용, 25건), 첫번째, 세,네번째 fold(트레이닝용, 25*3=75건) : 정확도\n세번째 fold(테스트용, 25건), 첫,두번째, 네번째 fold(트레이닝용, 25*3=75건) : 정확도\n네번째 fold(테스트용, 25건), 첫~세번째 fold(트레이닝용, 25*3=75건) :정확도\n\n정확도의 평균(최종 모델 정확도)\n\nkfold 교차검증\n'

In [177]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [178]:
kfold=KFold(n_splits=10, shuffle=True, random_state=42)

In [179]:
from sklearn.ensemble import RandomForestRegressor

In [180]:
model=RandomForestRegressor(n_estimators=100,
                     n_jobs=-1,
                     random_state=42)

In [181]:
fn=['season','holiday','workingday','weather',
            'year', 'hour', 'dayofweek', 'temp', 'atemp',
'humidity', 'windspeed' ]

In [182]:
xTrain=train[fn]
xTrain.shape #(10886, 12)

(10886, 11)

In [183]:
xTest=test[fn]
xTest.shape#(6493,12)

(6493, 11)

In [184]:
yTrain=train['count']
yTrain.shape #(10886,)

(10886,)

In [185]:
model.fit(xTrain, yTrain)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [186]:
pred=model.predict(xTest)
pred

array([ 12.52      ,   5.09      ,   3.71      , ...,  98.29      ,
       103.72333333,  46.22      ])

In [187]:
score=cross_val_score(model, xTrain, yTrain, cv=kfold, scoring=rmsle_scorer)
score
score.mean() #0.32

0.33111632569822946

In [188]:
bikesubmit=pd.read_csv('sampleSubmission.csv')
bikesubmit
bikesubmit['count']=pred
bikesubmit

bikesubmit.to_csv("bikesubmit.csv", index=False)

In [70]:
"""
for dataset in train_test:
    dataset.windspeed.replace(0,dataset.windspeed.mean(),inplace=True)
        
train['windspeed'][train.windspeed == 0] = train.windspeed.mean()

train['windspeed']=train['windspeed'].replace(0, np.nan)
train['windspeed']=train['windspeed'].fillna(train['windspeed'].mean())
"""

"\nfor dataset in train_test:\n    dataset.windspeed.replace(0,dataset.windspeed.mean(),inplace=True)\n        \ntrain['windspeed'][train.windspeed == 0] = train.windspeed.mean()\n\ntrain['windspeed']=train['windspeed'].replace(0, np.nan)\ntrain['windspeed']=train['windspeed'].fillna(train['windspeed'].mean())\n"